In [1]:
import hopsworks
import os
from dotenv import load_dotenv

import requests
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
from ta.trend import CCIIndicator

plt.style.use('grayscale')

# Load the .env file
load_dotenv()

# Get the environment variables
tiingo_api_key = os.getenv("TIINGO_API_KEY")
hopsworks_api_key = os.getenv("HOPSWORKS_API_KEY")


In [5]:
url = f"https://api.tiingo.com/iex/AMZN/prices?startDate=2025-02-19&resampleFreq=60min&token={tiingo_api_key}"
response = requests.get(url).json()
pd.DataFrame(response)

,date,close,high,low,open
0,2025-02-19T15:00:00.000Z,224.405,225.005,223.720,224.935
1,2025-02-19T16:00:00.000Z,224.965,225.000,224.220,224.410
2,2025-02-19T17:00:00.000Z,224.440,225.330,224.290,224.960
3,2025-02-19T18:00:00.000Z,224.800,225.085,224.005,224.445
4,2025-02-19T19:00:00.000Z,226.205,226.240,224.735,224.830
5,2025-02-19T20:00:00.000Z,226.650,226.830,225.850,226.205


In [23]:
def fetch_yesterdays_stock_data(ticker: str = "AMZN", freq: str = "60min", token: str | None = tiingo_api_key) -> pd.DataFrame:
    """
    Fetches stock data for a single ticker from the Tiingo API.

    Args:
        ticker (str): The stock ticker symbol. Default is "AMZN".
        freq (str): The frequency of the data. Default is "60min".
        token (str): The API token for authentication.

    Returns:
        pd.DataFrame: A DataFrame containing the stock data.
    """
    date = (datetime.now() - timedelta(1)).strftime("%Y-%m-%d")
	# Format the url with start_data, end_data and freq
    url = f"https://api.tiingo.com/iex/AMZN/prices?startDate={date}&resampleFreq={freq}&token={token}"
    response = requests.get(url).json()
    
	# Create DataFrame
    data = pd.DataFrame(response)
    
	# Remove the zone in date
    data['date'] = data['date'].map(lambda x: x.split('.')[0])
    data['date'] = pd.to_datetime(data['date'])
    data['id'] = data['date'].astype('str')
    return data

In [24]:
sample = fetch_yesterdays_stock_data(tiingo_api_key)
sample

,date,close,high,low,open,id
0,2025-02-19 15:00:00,224.405,225.005,223.720,224.935,2025-02-19 15:00:00
1,2025-02-19 16:00:00,224.965,225.000,224.220,224.410,2025-02-19 16:00:00
2,2025-02-19 17:00:00,224.440,225.330,224.290,224.960,2025-02-19 17:00:00
3,2025-02-19 18:00:00,224.800,225.085,224.005,224.445,2025-02-19 18:00:00
4,2025-02-19 19:00:00,226.205,226.240,224.735,224.830,2025-02-19 19:00:00
5,2025-02-19 20:00:00,226.650,226.830,225.850,226.205,2025-02-19 20:00:00


### Login to Hopsworks

In [25]:
project = hopsworks.login(api_key_value=str(hopsworks_api_key))

fs = project.get_feature_store()

# Get featuer group
amazon_fg = fs.get_feature_group("amazon_stock_prices", version=1)

2025-02-20 17:44:57,762 INFO: Initializing external client
2025-02-20 17:44:57,762 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-20 17:45:01,693 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212597


In [26]:
# Insert the data sample to feature group
amazon_fg.insert(sample)

Uploading Dataframe: 100.00% |██████████| Rows 6/6 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: amazon_stock_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212597/jobs/named/amazon_stock_prices_1_offline_fg_materialization/executions


(Job('amazon_stock_prices_1_offline_fg_materialization', 'SPARK'), None)